In [27]:
import pandas as pd
import numpy as np
import requests

## read the JSON file that you saved in ex02

In [28]:
df = pd.read_json("../ex02/data/auto.json")
df.set_index("CarNumber", inplace=True)
pd.options.display.float_format = '${:,.2f}'.format
df

,Refund,Fines,Make,Model
CarNumber,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus
E432XX77RUS,1,"$6,500.00",Toyota,Camry
7184TT36RUS,1,"$2,100.00",Ford,Focus
X582HE161RUS,2,"$2,000.00",Ford,Focus
92918M178RUS,1,"$5,700.00",Ford,Focus
...,...,...,...,...
Y163O8161RUS,2,"$1,600.00",Ford,Focus
M0309X197RUS,1,"$22,300.00",Ford,Focus
O673E8197RUS,2,$600.00,Ford,Focus


## enrich the dataframe using a sample from that dataframe

In [29]:
sample1 = df[['Make', 'Model']].sample(n=200, random_state=21)
sample2 = df[['Refund', 'Fines']].sample(n=200, replace=True, random_state=21)
sample1[['Refund', 'Fines']] = sample2[['Refund', 'Fines']].values
concat_rows = df.append(sample1)
concat_rows

,Refund,Fines,Make,Model
CarNumber,,,,
Y163O8161RUS,$2.00,"$3,200.00",Ford,Focus
E432XX77RUS,$1.00,"$6,500.00",Toyota,Camry
7184TT36RUS,$1.00,"$2,100.00",Ford,Focus
X582HE161RUS,$2.00,"$2,000.00",Ford,Focus
92918M178RUS,$1.00,"$5,700.00",Ford,Focus
...,...,...,...,...
8182XX154RUS,$1.00,"$2,000.00",Ford,Focus
X796TH96RUS,$2.00,$400.00,Ford,Focus
T011MY163RUS,$1.00,"$12,800.00",Ford,Focus


## enrich the dataframe concat_rows by a new column with the data generated

In [30]:
np.random.seed(21)
fines = concat_rows.copy()
ser = pd.Series(1980 + np.random.randint(0, 2019 - 1980,925 ), name="year")
ser = ser.to_frame()
fines[['year']] = ser[['year']].values
fines

,Refund,Fines,Make,Model,year
CarNumber,,,,,
Y163O8161RUS,$2.00,"$3,200.00",Ford,Focus,1989
E432XX77RUS,$1.00,"$6,500.00",Toyota,Camry,1995
7184TT36RUS,$1.00,"$2,100.00",Ford,Focus,1984
X582HE161RUS,$2.00,"$2,000.00",Ford,Focus,2015
92918M178RUS,$1.00,"$5,700.00",Ford,Focus,2014
...,...,...,...,...,...
8182XX154RUS,$1.00,"$2,000.00",Ford,Focus,1996
X796TH96RUS,$2.00,$400.00,Ford,Focus,2002
T011MY163RUS,$1.00,"$12,800.00",Ford,Focus,1996


## enrich the dataframe with the data from another dataframe

In [31]:
names = pd.read_json("surname.json")
names.rename(columns={0: names[0][0], 1: names[1][0], 2: names[2][0] }, inplace=True)
names = names.drop(labels=0, axis=0)
owners = names[['NAME']].sample(n=len(df.index.unique()), replace=True, random_state=21)
owners.index = df.index.unique()
owners.rename(columns={'NAME': 'SURENAME'}, inplace=True)
owners

,SURENAME
CarNumber,
Y163O8161RUS,RICHARDSON
E432XX77RUS,ROSS
7184TT36RUS,MORGAN
X582HE161RUS,BAILEY
92918M178RUS,LOPEZ
...,...
O136HO197RUS,CAMPBELL
O22097197RUS,HALL
M0309X197RUS,BAKER


In [32]:
rows = fines.sample(n=5)
rows.index.values[0] = 'X4128E125RUS'
rows.index.values[1] = 'X4128F125RUS'
rows.index.values[2] = 'X4128G125RUS'
rows.index.values[3] = 'X4128H125RUS'
rows.index.values[4] = 'X4128I125RUS'
fines = fines.append(rows)


In [33]:
owners = owners[:-20]
picked_names = names['NAME'].sample(n=3)
new_owners = owners['SURENAME'].sample(n=3)
new_owners.index.values[0] = 'Y652GD197RUS'
new_owners.index.values[1] = 'Y652GH197RUS'
new_owners.index.values[2] = 'Y652GJ197RUS'
new_owners[0] = list(picked_names)[0]
new_owners[1] = list(picked_names)[1]
new_owners[2] = list(picked_names)[2]
owners = owners.append(pd.DataFrame(new_owners))

In [34]:
#the new dataframe should have only the car numbers that exist in both dataframes
fines.merge(owners, left_index=True, right_index=True, how='inner')

,Refund,Fines,Make,Model,year,SURENAME
CarNumber,,,,,,
704687163RUS,$2.00,"$1,400.00",Ford,Focus,2014,ADAMS
704787163RUS,$2.00,"$2,800.00",Ford,Focus,2005,MORGAN
704987163RUS,$2.00,"$8,594.59",Ford,Focus,2014,MITCHELL
705287163RUS,$2.00,"$2,000.00",Ford,Focus,1990,GOMEZ
705387163RUS,$2.00,$700.00,Ford,Focus,2005,STEWART
...,...,...,...,...,...,...
Y973O8197RUS,$2.00,"$8,594.59",Ford,Focus,2005,YOUNG
Y973O8197RUS,$1.00,"$34,800.00",Ford,Focus,2013,YOUNG
Y973O8197RUS,$1.00,"$69,600.00",Ford,Focus,1989,YOUNG


In [35]:
#the new dataframe should have all the car numbers that exist in both dataframes
fines.merge(owners, left_index=True, right_index=True, how='outer')

,Refund,Fines,Make,Model,year,SURENAME
CarNumber,,,,,,
704687163RUS,$2.00,"$1,400.00",Ford,Focus,"$2,014.00",ADAMS
704787163RUS,$2.00,"$2,800.00",Ford,Focus,"$2,005.00",MORGAN
704987163RUS,$2.00,"$8,594.59",Ford,Focus,"$2,014.00",MITCHELL
705287163RUS,$2.00,"$2,000.00",Ford,Focus,"$1,990.00",GOMEZ
705387163RUS,$2.00,$700.00,Ford,Focus,"$2,005.00",STEWART
...,...,...,...,...,...,...
Y973O8197RUS,$2.00,"$8,594.59",Ford,Focus,"$2,005.00",YOUNG
Y973O8197RUS,$1.00,"$34,800.00",Ford,Focus,"$2,013.00",YOUNG
Y973O8197RUS,$1.00,"$69,600.00",Ford,Focus,"$1,989.00",YOUNG


In [36]:
#the new dataframe should have only the car numbers from the fines dataframe
pd.merge(fines, owners, left_index=True, right_index=True, how='left')

,Refund,Fines,Make,Model,year,SURENAME
CarNumber,,,,,,
704687163RUS,$2.00,"$1,400.00",Ford,Focus,2014,ADAMS
704787163RUS,$2.00,"$2,800.00",Ford,Focus,2005,MORGAN
704987163RUS,$2.00,"$8,594.59",Ford,Focus,2014,MITCHELL
705287163RUS,$2.00,"$2,000.00",Ford,Focus,1990,GOMEZ
705387163RUS,$2.00,$700.00,Ford,Focus,2005,STEWART
...,...,...,...,...,...,...
Y973O8197RUS,$2.00,"$8,594.59",Ford,Focus,2005,YOUNG
Y973O8197RUS,$1.00,"$34,800.00",Ford,Focus,2013,YOUNG
Y973O8197RUS,$1.00,"$69,600.00",Ford,Focus,1989,YOUNG


In [37]:
pd.merge(fines, owners, left_index=True, right_index=True, how='right')

,Refund,Fines,Make,Model,year,SURENAME
CarNumber,,,,,,
704687163RUS,$2.00,"$1,400.00",Ford,Focus,"$2,014.00",ADAMS
704787163RUS,$2.00,"$2,800.00",Ford,Focus,"$2,005.00",MORGAN
704987163RUS,$2.00,"$8,594.59",Ford,Focus,"$2,014.00",MITCHELL
705287163RUS,$2.00,"$2,000.00",Ford,Focus,"$1,990.00",GOMEZ
705387163RUS,$2.00,$700.00,Ford,Focus,"$2,005.00",STEWART
...,...,...,...,...,...,...
Y973O8197RUS,$2.00,"$8,594.59",Ford,Focus,"$2,005.00",YOUNG
Y973O8197RUS,$1.00,"$34,800.00",Ford,Focus,"$2,013.00",YOUNG
Y973O8197RUS,$1.00,"$69,600.00",Ford,Focus,"$1,989.00",YOUNG


In [38]:
'Y652GH197RUS' in pd.merge(fines, owners, left_index=True, right_index=True, how='right').index.values #owner number

True

In [39]:
'X4128I125RUS' in pd.merge(fines, owners, left_index=True, right_index=True, how='right').index.values #fines number

False

## create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but with all the years (the values may be different for you):

In [40]:
table = pd.pivot_table(fines, values='Fines', index=['Make', 'Model'],
                    columns=['year'], aggfunc=np.sum)
table

year                     1980        1981       1982        1983        1984  \
Make       Model                                                               
Ford       Focus   $74,994.59 $306,394.59 $88,994.59 $163,994.59 $112,500.00   
           Mondeo         NaN         NaN $46,200.00         NaN         NaN   
Skoda      Octavia  $9,094.59   $1,900.00  $8,894.59         NaN     $500.00   
Toyota     Camry   $12,000.00         NaN $43,600.00   $1,000.00   $1,000.00   
           Corolla        NaN   $6,800.00        NaN  $12,800.00         NaN   
Volkswagen Golf    $20,800.00   $8,594.59  $5,000.00     $200.00         NaN   
           Jetta          NaN   $1,000.00        NaN         NaN         NaN   
           Passat     $900.00   $4,000.00        NaN   $1,100.00   $8,594.59   
           Touareg        NaN         NaN        NaN         NaN         NaN   

year                      1985       1986        1987       1988        1989  \
Make       Model                                                               
Ford       Focus   $262,694.59 $62,200.00 $161,494.59 $77,772.93 $168,594.59   
           Mondeo          NaN        NaN         NaN        NaN         NaN   
Skoda      Octavia  $10,894.59        NaN  $22,600.00  $5,100.00   $8,594.59   
Toyota     Camry           NaN $19,800.00         NaN        NaN     $800.00   
           Corolla   $6,300.00        NaN  $54,300.00        NaN   $7,800.00   
Volkswagen Golf    $168,000.00        NaN     $500.00        NaN     $300.00   
           Jetta     $9,000.00        NaN         NaN $46,000.00     $500.00   
           Passat          NaN $16,000.00   $2,000.00  $8,594.59         NaN   
           Touareg         NaN        NaN         NaN        NaN         NaN   

year                ...        2009        2010        2011        2012  \
Make       Model    ...                                                   
Ford       Focus    ... $147,594.59 $111,100.00 $116,994.59 $143,789.17   
           Mondeo   ...         NaN         NaN         NaN         NaN   
Skoda      Octavia  ...         NaN   $4,000.00   $3,000.00   $1,700.00   
Toyota     Camry    ...         NaN  $22,400.00         NaN   $7,500.00   
           Corolla  ...   $8,594.59   $6,000.00   $1,000.00         NaN   
Volkswagen Golf     ...         NaN         NaN         NaN         NaN   
           Jetta    ...         NaN         NaN         NaN         NaN   
           Passat   ...  $21,800.00   $9,500.00         NaN     $800.00   
           Touareg  ...   $5,800.00         NaN         NaN         NaN   

year                      2013       2014        2015        2016        2017  \
Make       Model                                                                
Ford       Focus   $292,183.76 $74,083.76 $171,500.00 $102,789.17 $102,894.59   
           Mondeo   $41,100.00        NaN         NaN         NaN   $8,600.00   
Skoda      Octavia  $11,800.00 $52,600.00  $16,394.59  $35,700.00   $2,400.00   
Toyota     Camry           NaN        NaN         NaN   $8,594.59         NaN   
           Corolla         NaN        NaN   $7,500.00         NaN   $9,200.00   
Volkswagen Golf            NaN $13,900.00   $5,300.00         NaN         NaN   
           Jetta           NaN        NaN         NaN         NaN         NaN   
           Passat    $1,600.00  $3,800.00         NaN         NaN         NaN   
           Touareg         NaN        NaN         NaN         NaN         NaN   

year                      2018  
Make       Model                
Ford       Focus    $99,500.00  
           Mondeo          NaN  
Skoda      Octavia $153,200.00  
Toyota     Camry           NaN  
           Corolla         NaN  
Volkswagen Golf      $5,000.00  
           Jetta           NaN  
           Passat          NaN  
           Touareg         NaN  

[9 rows x 39 columns]

## save both the fines and owners dataframes to CSV files without an index

In [41]:
fines.reset_index(inplace=True)
owners.reset_index(inplace=True)
fines.to_csv('./data/fines.csv', index=False)
owners.to_csv('./data/owners.csv', index=False)